# Layout

The first step is to create a layout and test manifest for your project, so that you can relate measurements and analyses back to your source layout.

You can generate the layout with any layout tool. In this notebook you can use gdsfactory.

## Generate layout

You can use [generate_layout.py](./generate_layout.py) to create a GDS layout file with different rings.

In [ ]:
from generate_layout import top

c = top()
c.write_gds("test_chip.gds")
c.plot()

## Generate device table

A design manifest is a table of devices and settings. A device contains the cell id, (x, y) locations and settings.

It can also optionally contain `analysis` and `analysis_parameters` that will trigger automated analysis when uploading new data associated to the device.

Here you can read them from the GDS and write a design manifest, which we can use to associate measurement data with the devices on the GDS. However, you can use any method you prefer to generate your test manifest.

In [ ]:
import csv

import kfactory as kf

c = kf.kcl["top"]
rings = c.kcl["rings"]
csvpath = "design_manifest.csv"

analysis = "[fsr]"
analysis_parameters = '[{"height": -0.01, "distance": 20}]'

with open(csvpath, "w") as f:
    writer = csv.writer(f)
    writer.writerow(
        [
            "cell",
            "x",
            "y",
            "radius_um",
            "gap_um",
            "analysis",
            "analysis_parameters",
        ]
    )

    iterator = rings.begin_instances_rec()
    iterator.targets = "Ring*"
    while not iterator.at_end():
        _c = c.kcl[iterator.inst_cell().cell_index()]
        name = _c.name
        if "RingDouble" in name:
            _disp = (iterator.trans() * iterator.inst_trans()).disp
            params = name.split("-")
            writer.writerow(
                [
                    name,
                    _disp.x,
                    _disp.y,
                    int(params[1]),
                    float(params[2]),
                    analysis,
                    analysis_parameters,
                ]
            )
        iterator.next()

You can take a look a the contents of the device manifest you created.

In [ ]:
import pandas as pd

df = pd.read_csv(csvpath)
df